In [1]:
# Importa librerías
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader

In [2]:
# Verifica la GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")
print(f"Versión de PyTorch: {torch.__version__}")
print(f"Versión de CUDA: {torch.version.cuda}")
print(f"Memoria GPU disponible: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

Usando dispositivo: cuda
Versión de PyTorch: 2.8.0+cu126
Versión de CUDA: 12.6
Memoria GPU disponible: 4.00 GB


In [3]:
# Carga los datos
data = pd.read_csv('../data/processed/physical_exercise_enriched.csv')
print("Primeras 5 filas del dataset:")
print(data.head())

Primeras 5 filas del dataset:
   pose_id    x_nose  x_left_eye_inner  x_left_eye  x_left_eye_outer  \
0      659  0.005001          0.011026    0.010531          0.017030   
1      660  0.005643          0.009842    0.009092          0.018570   
2      661 -0.080741         -0.111531   -0.083575         -0.111369   
3      662  0.011821          0.024027    0.020280          0.030158   
4      663 -0.004911         -0.006103    0.000368          0.007008   

   x_right_eye_inner  x_right_eye  x_right_eye_outer  x_left_ear  x_right_ear  \
0           0.000794    -0.002900          -0.005019    0.022900    -0.011029   
1           0.001104    -0.001759          -0.003827    0.018725    -0.009088   
2          -0.083769    -0.111719          -0.083855   -0.107371    -0.081137   
3           0.011744     0.013145           0.008056    0.035715     0.004197   
4          -0.019879    -0.032767          -0.029256    0.020583    -0.035129   

   ...  y_right_hip  y_left_knee  y_right_knee  y_

Preparación de los datos

Vamos a convertir los datos en secuencias temporales para el LSTM, ya que los ejercicios (e.g., pushups_down/up, squats_down/up) tienen un componente temporal.

In [4]:
# Preprocesamiento de datos
def create_sequences(data, seq_length=10):
    sequences = []
    targets = []
    for i in range(len(data) - seq_length):
        seq = data.iloc[i:i + seq_length][['x_nose', 'y_nose', 'x_left_shoulder', 'y_left_shoulder', 
                                          'x_right_shoulder', 'y_right_shoulder', 'x_left_elbow', 'y_left_elbow',
                                          'x_right_elbow', 'y_right_elbow', 'x_left_wrist', 'y_left_wrist',
                                          'x_right_wrist', 'y_right_wrist', 'x_left_hip', 'y_left_hip',
                                          'x_right_hip', 'y_right_hip', 'x_left_knee', 'y_left_knee',
                                          'x_right_knee', 'y_right_knee', 'x_left_ankle', 'y_left_ankle',
                                          'x_right_ankle', 'y_right_ankle', 'right_elbow_angle',
                                          'left_elbow_angle', 'right_knee_angle', 'left_knee_angle']].values
        target = data.iloc[i + seq_length]['pose']  # Ahora debería existir
        sequences.append(seq)
        targets.append(target)
    return np.array(sequences), np.array(targets)

# Crea secuencias
seq_length = 10  # Longitud de la secuencia (ajustable)
sequences, targets = create_sequences(data, seq_length)

# Codifica las etiquetas
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
targets_encoded = le.fit_transform(targets)
print(f"Etiquetas únicas: {le.classes_}")
print(f"Forma de las secuencias: {sequences.shape}, Forma de los targets: {targets_encoded.shape}")

# Crea un Dataset personalizado
class ExerciseDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = torch.FloatTensor(sequences)
        self.targets = torch.LongTensor(le.transform(targets))  # Usa transform para consistencia
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return self.sequences[idx], self.targets[idx]

dataset = ExerciseDataset(sequences, targets)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

Etiquetas únicas: ['pushups_down' 'pushups_up' 'situp_down' 'situp_up' 'squats_down'
 'squats_up']
Forma de las secuencias: (14620, 10, 30), Forma de los targets: (14620,)


Define y entrena el modelo LSTM.

In [5]:
# Define el modelo LSTM
class LSTMModel(nn.Module):
    def __init__(self, input_size=30, hidden_size=64, num_layers=2, num_classes=6):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Inicializa el modelo
input_size = 30  # 26 keypoints (x, y) + 4 ángulos
hidden_size = 64
num_layers = 2
num_classes = len(le.classes_)  # Número de etiquetas únicas
model = LSTMModel(input_size, hidden_size, num_layers, num_classes).to(device)
print(model)

# Define loss y optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Entrena el modelo
num_epochs = 50
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print("Entrenamiento completado.")

LSTMModel(
  (lstm): LSTM(30, 64, num_layers=2, batch_first=True)
  (fc): Linear(in_features=64, out_features=6, bias=True)
)
Epoch [10/50], Loss: 0.0311
Epoch [20/50], Loss: 0.1202
Epoch [30/50], Loss: 0.0909
Epoch [40/50], Loss: 0.0084
Epoch [50/50], Loss: 0.0383
Entrenamiento completado.


guardar el modelo y realizar una evaluación básica

In [6]:
# Guarda el modelo
torch.save(model.state_dict(), '../models/lstm_modelV2.pth')
print("Modelo guardado en ../models/lstm_modelV2.pth")

# Evaluación básica (pérdida en el conjunto de entrenamiento)
model.eval()
total_loss = 0
with torch.no_grad():
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        total_loss += loss.item()

avg_loss = total_loss / len(dataloader)
print(f"Pérdida promedio en el conjunto de entrenamiento: {avg_loss:.4f}")

# (Opcional) Predicción en un ejemplo
sample_idx = 0
sample_input, sample_label = dataset[sample_idx]
sample_input = sample_input.unsqueeze(0).to(device)  # Añade dimensión de batch
with torch.no_grad():
    sample_output = model(sample_input)
    _, predicted = torch.max(sample_output, 1)
    print(f"Etiqueta real: {le.inverse_transform([sample_label.cpu().numpy()])[0]}")
    print(f"Etiqueta predicha: {le.inverse_transform([predicted.cpu().numpy()])[0]}")

Modelo guardado en ../models/lstm_modelV2.pth
Pérdida promedio en el conjunto de entrenamiento: 0.0584
Etiqueta real: pushups_down
Etiqueta predicha: pushups_down


c:\Users\CHRISTIAN\Documents\EvaluacionEjerciciosIA\.venv\lib\site-packages\sklearn\preprocessing\_label.py:151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
